In [33]:
import sys
sys.path.append('../')

import torch
from transformers import BertTokenizer, BertModel
from utils.process_data import get_dataloader
import torch.optim as optim
from utils import config
import numpy as np


class BertCls(torch.nn.Module):
    def __init__(self):
        super(BertCls, self).__init__()
        self.bert = BertModel.from_pretrained(config.bert_model_path)
        self.liner = torch.nn.Sequential(
            torch.nn.Linear(768 * 2, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 1),
            torch.nn.Sigmoid()
        )

    def forward(self, ste1, ste2):
        ebd1, cls1 = self.bert(ste1)
        ebd2, cls2 = self.bert(ste2)
        conact = torch.cat((ebd1[:, 0, :], ebd2[:, 0, :]), dim=1)
        out = self.liner(conact)
        return out


def freeze_parameter(cls_model):
    for n,p in cls_model.named_parameters():
        if 'bert' in n:
            p.requires_grad = False
    for n,p in cls_model.named_parameters():
        if 'bert.encoder.layer.11' in n:
            p.requires_grad = True
    
def train(model, train_data, epco=10):
    loss_fn = torch.nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    
    for e in range(epco):
        idx = 0
        for s1, s2, l in train_data:
            optimizer.zero_grad()
            y = model(s1, s2)
            loss = loss_fn(y, l)
            loss.backward()
            optimizer.step()

            if idx % 10 == 9:
                print('epco:{} iter:{} loss:{}'.format(e, idx, loss))
            idx += 1


def evaluate(model, test_data):
    model.eval()
    right = 0.1
    preidt_p = 0.1
    positive = 0.1
    with torch.no_grad():
        for s1, s2, l in test_data:
            y = model(s1, s2)
            y = y.cpu().view(-1).numpy()
            y[y > 0.5] = 1
            y[y <= 0.5] = 0
            preidt_p += y.sum()
            
            l = l.cpu().view(-1).numpy()
            positive += l.sum()
            l[l==0]=-1
            right += (y == l).sum()
    P = right / preidt_p
    R = right / positive
    F1 = 2 * P * R / (P + R)
    print('P:{} R:{} F1:{}'.format(P, R, F1))

In [26]:
train_data, test_data = get_dataloader()
cls_model = BertCls()
cls_model.cuda()
freeze_parameter(cls_model)
train(cls_model, train_data)
evaluate(cls_model, test_data)

epco:0 iter:9 loss:0.7578407526016235
epco:0 iter:19 loss:0.6944000720977783
epco:0 iter:29 loss:0.6681841611862183
epco:0 iter:39 loss:0.6725636720657349
epco:0 iter:49 loss:0.6306008696556091
epco:0 iter:59 loss:0.6407224535942078
epco:1 iter:9 loss:0.5894600749015808
epco:1 iter:19 loss:0.6995288729667664
epco:1 iter:29 loss:0.607067346572876
epco:1 iter:39 loss:0.5976476669311523
epco:1 iter:49 loss:0.6352135539054871
epco:1 iter:59 loss:0.5339100956916809
epco:2 iter:9 loss:0.577693521976471
epco:2 iter:19 loss:0.42934751510620117
epco:2 iter:29 loss:0.43866056203842163
epco:2 iter:39 loss:0.5503401756286621
epco:2 iter:49 loss:0.6623993515968323
epco:2 iter:59 loss:0.5927455425262451
epco:3 iter:9 loss:0.6484025716781616
epco:3 iter:19 loss:0.4324478507041931
epco:3 iter:29 loss:0.6393012404441833
epco:3 iter:39 loss:0.5291118025779724
epco:3 iter:49 loss:0.5391762256622314
epco:3 iter:59 loss:0.5937991738319397
epco:4 iter:9 loss:0.6072697043418884
epco:4 iter:19 loss:0.48190221

P:0.6399871423979427 R:0.8432867429055485 F1:0.7277046783625731


In [28]:
import numpy as np
a=np.random.rand(32)
a[a>0.5]=1
a[a<=0.5]=0
b=np.random.rand(32)
b[b>0.5]=1
b[b<=0.5]=0

In [31]:
a==1

array([False, False,  True, False,  True, False, False,  True,  True,
        True, False,  True, False,  True, False, False,  True, False,
        True, False,  True, False, False,  True, False,  True,  True,
        True,  True, False, False,  True])

In [ ]:
a==1